In [83]:
from collections import defaultdict
from joblib import Parallel, delayed
import pandas as pd
from tqdm import tqdm
import os

In [84]:
import jams

In [85]:
print(jams.__version__)

0.2.1


In [86]:
def merge_annotations(upper=None, lower=None):
    
    ann = jams.Annotation(namespace='multi_segment',
                          annotation_metadata=upper.annotation_metadata)
    
    try:
        ann.sandbox.update(**upper.sandbox)
    except AttributeError:
        pass
    
    try:
        ann.sandbox.update(**lower.sandbox)
    except AttributeError:
        pass
    
    # Add the uppers
    d_upper = upper.data.copy()
    d_upper['value'] = [{'label': _, 'level': 0} for _ in d_upper['value']]
    
    # Add the lowers
    d_lower = lower.data.copy()
    d_lower['value'] = [{'label': _, 'level': 1} for _ in d_lower['value']]
    
    ann.data = jams.JamsFrame.from_dataframe(pd.concat([d_upper, d_lower], ignore_index=True))
    
    return ann
    
    
def convert_jams(jamsfile):
    
    J = jams.load(jamsfile)
    
    ann_dict = defaultdict(dict)
    
    for upper in J.search(namespace='segment_salami_upper'):
        name = upper.annotation_metadata.annotator.name
        ann_dict[name]['upper'] = upper
        
    for lower in J.search(namespace='segment_salami_lower'):
        name = lower.annotation_metadata.annotator.name
        ann_dict[name]['lower'] = lower
    
    for key in ann_dict:
        J.annotations.append(merge_annotations(**ann_dict[key]))
        
    return J

In [88]:
files = jams.util.find_with_extension('/home/bmcfee/git/msaf-data/SPAM/references/', 'jams')
outdir = '/home/bmcfee/git/segment_hierarchy_labels/data/'

In [90]:
for fn in tqdm(files):
    J = convert_jams(fn)
    J.save(os.path.join(outdir, os.path.basename(fn)))